In [1]:
"""Prepares the input pipeline for OpenWebText (OWT).

This script tokenizes the OWT dataset and splits it into train and eval sets.
The train and eval sets are saved as ArrayRecord files.
"""

# from array_record.python import array_record_module
import datasets

# import numpy as np
# import tensorflow as tf
# import tqdm
import transformers

# import time


source = datasets.load_dataset(
    "Skylion007/openwebtext", name="plain_text", split="train", streaming=True
)

_GPT2_TOKENIZER = "gpt2"
tokenizer = transformers.GPT2Tokenizer.from_pretrained(_GPT2_TOKENIZER)

# ArrayRecordWriter = array_record_module.ArrayRecordWriter
# ArrayRecordReader = array_record_module.ArrayRecordReader


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


ds_output_file_train = "./data_dir/openwebtext_splits_1024_train"
ds_output_file_eval = "./data_dir/openwebtext_splits_1024_eval"

/home/ubuntu/anaconda3/envs/md4-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
from transformers import AutoTokenizer
from joblib import Parallel, delayed

In [3]:
def tokenize_chunk(chunk):
    return tokenizer(chunk)


# Main function
def parallel_tokenize(strings, chunk_size=1000, num_jobs=-1):
    """
    Tokenizes a large list of strings in parallel.

    Args:
        strings (list of str): The list of strings to tokenize.
        chunk_size (int): The size of each chunk for processing.
        num_jobs (int): The number of parallel jobs (-1 uses all CPUs).

    Returns:
        dict: A dictionary with tokenized results combined across all chunks.
    """
    # Split data into chunks
    chunks = [strings[i : i + chunk_size] for i in range(0, len(strings), chunk_size)]

    # Tokenize each chunk in parallel
    tokenized_chunks = Parallel(n_jobs=num_jobs)(
        delayed(tokenize_chunk)(chunk) for chunk in chunks
    )

    return tokenized_chunks

In [4]:
block_size = 1024  # size of the chunk

data_iter = iter(source)

In [6]:
all_tokens = []
count = 0
count_per_save = 0
eval_chunks = []

from tqdm import tqdm

text_list = []
for i, example in tqdm(enumerate(data_iter)):
    text_list.append(example["text"])
    if i >= 100 * 512:
        tokenized_result = parallel_tokenize(text_list, chunk_size=100, num_jobs=512)
        break
    continue
    tokens = tokenizer(example["text"])["input_ids"]
    all_tokens.extend(tokens + [tokenizer.eos_token_id])

51193it [00:30, 3546.37it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (1025 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1846 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1253 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1164 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1493 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the spe

In [ ]:
tokenized_result = parallel_tokenize(text_list, chunk_size=1000, num_jobs=512)

In [13]:
text_list[0]

'Port-au-Prince, Haiti (CNN) -- Earthquake victims, writhing in pain and grasping at life, watched doctors and nurses walk away from a field hospital Friday night after a Belgian medical team evacuated the area, saying it was concerned about security.\n\nThe decision left CNN Chief Medical Correspondent Sanjay Gupta as the only doctor at the hospital to get the patients through the night.\n\nCNN initially reported, based on conversations with some of the doctors, that the United Nations ordered the Belgian First Aid and Support Team to evacuate. However, Belgian Chief Coordinator Geert Gijs, a doctor who was at the hospital with 60 Belgian medical personnel, said it was his decision to pull the team out for the night. Gijs said he requested U.N. security personnel to staff the hospital overnight, but was told that peacekeepers would only be able to evacuate the team.\n\nHe said it was a "tough decision" but that he accepted the U.N. offer to evacuate after a Canadian medical team, also

In [ ]:
import transformers

_GPT2_TOKENIZER = "gpt2"
tokenizer = transformers.GPT2Tokenizer.from_pretrained(_GPT2_TOKENIZER)

/home/ubuntu/anaconda3/envs/lean-dojo/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
tokenizer.decode([13])

'.'

In [7]:
import datasets
from transformers import GPT2Tokenizer
from multiprocessing import Pool
import os

# Constants
_GPT2_TOKENIZER = "gpt2"
CHUNK_SIZE = 100
NUM_WORKERS = os.cpu_count()  # Number of workers (adjust as needed)

# Load the dataset (streaming mode)
source = datasets.load_dataset(
    "Skylion007/openwebtext", name="plain_text", split="train"
)


def process_chunk(chunk_range):
    """
    Tokenize a chunk of data using skip and take for streaming datasets.

    Args:
        chunk_range (tuple): Start and end indices for the chunk.

    Returns:
        list of dict: Tokenized data.
    """
    start, end = chunk_range
    tokenizer = GPT2Tokenizer.from_pretrained(_GPT2_TOKENIZER)

    # Efficiently access the desired range
    chunk = source.skip(start).take(end - start)
    tokenized_data = [tokenizer(text["text"]) for text in chunk]

    return tokenized_data


def create_chunks(total_size, chunk_size):
    """Create chunk ranges for parallel processing."""
    return [
        (i, min(i + chunk_size, total_size)) for i in range(0, total_size, chunk_size)
    ]


def save_tokenized_data(tokenized_data, output_file):
    """Save tokenized data to a file."""
    with open(output_file, "w") as f:
        for item in tokenized_data:
            f.write(str(item) + "\n")


if __name__ == "__main__":
    total_size = 100000  # Adjust to the number of samples in your dataset
    chunks = create_chunks(total_size, CHUNK_SIZE)

    # Run parallel tokenization
    with Pool(NUM_WORKERS) as pool:
        tokenized_results = pool.map(process_chunk, chunks)

    # Combine and save results
    all_tokenized_data = [item for sublist in tokenized_results for item in sublist]
    # save_tokenized_data(all_tokenized_data, "./data_dir/openwebtext_tokenized.txt")
    import pickle

    output_pickle_file = "./data_dir/openwebtext_tokenized.pkl"
    with open(output_pickle_file, "wb") as f:
        pickle.dump(all_tokenized_data, f)

    print(f"Tokenization complete. Total tokenized entries: {len(all_tokenized_data)}")

Generating train split:   1%|          | 71276/8013769 [00:15<27:53, 4744.70 examples/s]


KeyboardInterrupt: 

In [14]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt")

NameError: name 'GPT2Tokenizer' is not defined

In [2]:
os.cpu_count()

240

In [9]:
from datasets import load_from_disk

In [10]:
source = load_from_disk("./openwebtext_cache")

In [11]:
source[]

Dataset({
    features: ['text'],
    num_rows: 8013769
})

In [ ]:
import pickle

output_pickle_file = "./openwebtext_tokenized.pkl"
with open(output_pickle_file, "rb") as f:
    all_tokenized_data = pickle.load(f)

/home/ubuntu/anaconda3/envs/md4-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fecf6ca1db0>>
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/md4-venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [ ]:
all_tokenized_data_input_ids = [data["input_ids"] for data in all_tokenized_data]

In [13]:
all_tokenized_data_input_ids[-2][-1]

13

In [1]:
import pickle
import numpy as np

output_pickle_file = "./openwebtext_tokenized_inputids.pkl"
with open(output_pickle_file, "rb") as f:
    all_tokenized_data_input_ids = pickle.load(f)

In [2]:
len(all_tokenized_data_input_ids)

8013769

In [ ]:
import pickle
import numpy as np

output_pickle_file = "./openwebtext_tokenized_inputids.pkl"
with open(output_pickle_file, "rb") as f:
    all_tokenized_data_input_ids = pickle.load(f)

# ArrayRecordWriter = array_record_module.ArrayRecordWriter
# ArrayRecordReader = array_record_module.ArrayRecordReader


# def _int64_feature(value):
#     """Returns an int64_list from a bool / enum / int / uint."""
#     return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
import transformers
import numpy as np

_GPT2_TOKENIZER = "gpt2"
tokenizer = transformers.GPT2Tokenizer.from_pretrained(_GPT2_TOKENIZER)

ds_output_file_train = "./data_dir/openwebtext_splits_1024_train"
ds_output_file_eval = "./data_dir/openwebtext_splits_1024_eval"

n_examples = 8013769  # tiny: 2; small: 10_000; full: 8013769
save_every_examples = 100_000
block_size = 1024  # size of the chunk

# data_iter = iter(source)

all_tokens = []
count = 0
count_per_save = 0
eval_chunks = []

# writer_train = ArrayRecordWriter(ds_output_file_train, "group_size:1")
# writer_eval = ArrayRecordWriter(ds_output_file_eval, "group_size:1")

from tqdm import tqdm

for tokens in tqdm(all_tokenized_data_input_ids):
    # tokens = tokenizer(example["text"])["input_ids"]
    all_tokens.extend(tokens + [tokenizer.eos_token_id])
    count += 1
    count_per_save += 1
    
all_tokens = np.array(all_tokens)
np.save("openwebtext_tokenized_inputids.npy", all_tokens)

/home/ubuntu/anaconda3/envs/md4-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 8013769/8013769 [03:41<00:00, 36182.86it/s]


In [5]:
all_tokens = np.array(all_tokens)
np.save("openwebtext_tokenized_inputids.npy", all_tokens)

In [4]:
import numpy as np

all_tokens = np.array(all_tokens)

In [7]:
saved_length = (len(all_tokens) // block_size) * block_size
all_tokens_short = all_tokens[:saved_length]

In [9]:
all_tokens_short=all_tokens_short.reshape(-1, block_size)

In [11]:
chunks = all_tokens_short
np.random.shuffle(chunks)
num_eval = int(len(chunks) * 0.02)  # put 2% of chunks into eval split.

In [12]:
eval_chunks = chunks[:num_eval]
train_chunks = chunks[num_eval:]

In [13]:
# np.save("./data_dir/openwebtext_np_train.npy", train_chunks)
# np.save("./data_dir/openwebtext_np_eval.npy", eval_chunks)

In [15]:
train_chunks[0]

array([  257,  1256,   286, ..., 26318,  1757,   371])

In [16]:
train_chunks[1]

array([ 389, 1016,  284, ...,   64,  319, 3909])

In [6]:
import numpy as np

saved_length = (len(all_tokens) // block_size) * block_size
chunks = [
    all_tokens[i : i + block_size] for i in tqdm(range(0, saved_length, block_size))
]

# print("Time taken to tokenize:", time.time() - time1)
print(f"\nsaving chunks @ {count}th example mark...")
np.random.shuffle(chunks)
num_eval = int(len(chunks) * 0.02)  # put 2% of chunks into eval split.

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x79ea1752ddb0>>
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/md4-venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


: 

: 

In [3]:
from array_record.python import array_record_module
import datasets
import numpy as np
import tensorflow as tf
import tqdm
import transformers
import time
import os
from datasets import load_from_disk

/home/ubuntu/anaconda3/envs/md4-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-18 08:08:44.027518: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737187724.068846 1569184 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737187724.080157 1569184 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-18 08:08:44.113937: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-

In [6]:
_GPT2_TOKENIZER = "gpt2"
tokenizer = transformers.GPT2Tokenizer.from_pretrained(_GPT2_TOKENIZER)

In [14]:
ArrayRecordWriter = array_record_module.ArrayRecordWriter
ArrayRecordReader = array_record_module.ArrayRecordReader


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


ds_output_file_train = "./data_dir/openwebtext_splits_1024_train"
ds_output_file_eval = "./data_dir/openwebtext_splits_1024_eval"

n_examples = 8013769  # tiny: 2; small: 10_000; full: 8013769
save_every_examples = 100_000
block_size = 1024  # size of the chunk

# data_iter = iter(source)

all_tokens = []
count = 0
count_per_save = 0
eval_chunks = []

writer_train = ArrayRecordWriter(ds_output_file_train, "group_size:1")
writer_eval = ArrayRecordWriter(ds_output_file_eval, "group_size:1")

from tqdm import tqdm

for tokens in tqdm(all_tokenized_data_input_ids):
    # tokens = tokenizer(example["text"])["input_ids"]
    all_tokens.extend(tokens + [tokenizer.eos_token_id])
    count += 1
    count_per_save += 1

    # pause to save when having tokenized enough examples for saving.
    time1 = time.time()
    if count_per_save >= save_every_examples:
        # save to disk
        saved_length = (len(all_tokens) // block_size) * block_size
        chunks = [
            all_tokens[i : i + block_size] for i in range(0, saved_length, block_size)
        ]

        print("Time taken to tokenize:", time.time() - time1)
        print(f"\nsaving chunks @ {count}th example mark...")
        np.random.shuffle(chunks)
        num_eval = int(len(chunks) * 0.02)  # put 2% of chunks into eval split.
        for eval_i in tqdm(range(num_eval)):
            feature = {
                "text": _int64_feature(chunks[eval_i]),
            }
            example_proto = tf.train.Example(
                features=tf.train.Features(feature=feature)
            )
            writer_eval.write(example_proto.SerializeToString())

        for train_i in tqdm(range(num_eval, len(chunks))):
            feature = {
                "text": _int64_feature(chunks[train_i]),
            }
            example_proto = tf.train.Example(
                features=tf.train.Features(feature=feature)
            )
            writer_train.write(example_proto.SerializeToString())
        print("Time taken to save:", time.time() - time1)
        # prepare for the next round of tokenize-n-save.
        all_tokens = all_tokens[saved_length:]
        count_per_save = 0

    # stop when having tokenized enough examples for total #.
    if count >= n_examples:
        # save to disk
        saved_length = (len(all_tokens) // block_size) * block_size
        chunks = [
            all_tokens[i : i + block_size] for i in range(0, saved_length, block_size)
        ]

        print(f"\nsaving chunks @ {count}th example mark...")
        np.random.shuffle(chunks)
        num_eval = int(len(chunks) * 0.02)  # put 2% of chunks into eval split.
        for eval_i in tqdm(range(num_eval)):
            feature = {
                "text": _int64_feature(chunks[eval_i]),
            }
            example_proto = tf.train.Example(
                features=tf.train.Features(feature=feature)
            )
            writer_eval.write(example_proto.SerializeToString())

        for train_i in tqdm(range(num_eval, len(chunks))):
            feature = {
                "text": _int64_feature(chunks[train_i]),
            }
            example_proto = tf.train.Example(
                features=tf.train.Features(feature=feature)
            )
            writer_train.write(example_proto.SerializeToString())
        break

writer_train.close()
writer_eval.close()

  0%|          | 0/8013769 [00:00<?, ?it/s]

  1%|          | 96461/8013769 [00:02<03:07, 42165.58it/s]

Time taken to tokenize: 2.49033784866333

saving chunks @ 100000th example mark...


  1%|▏         | 108634/8013769 [00:40<2:56:11, 747.78it/s]

Time taken to save: 37.19849228858948


...

100%|█████████▉| 8007351/8013769 [58:11<00:09, 668.11it/s]  

Time taken to save: 34.462011098861694


100%|█████████▉| 8011000/8013769 [58:11<00:02, 957.70it/s]


saving chunks @ 8013769th example mark...


100%|█████████▉| 8013768/8013769 [58:16<00:00, 2291.79it/s]


In [8]:
tokens

(0,
 [13924,
  12,
  559,
  12,
  35784,
  11,
  25051,
  357,
  18474,
  8,
  1377,
  45591,
  4970,
  11,
  1319,
  44556,
  287,
  2356,
  290,
  44787,
  379,
  1204,
  11,
  7342,
  7519,
  290,
  20669,
  2513,
  1497,
  422,
  257,
  2214,
  4436,
  3217,
  1755,
  706,
  257,
  21402,
  3315,
  1074,
  23724,
  262,
  1989,
  11,
  2282,
  340,
  373,
  5213,
  546,
  2324,
  13,
  198,
  198,
  464,
  2551,
  1364,
  8100,
  5953,
  8366,
  34428,
  298,
  2986,
  33708,
  42095,
  355,
  262,
  691,
  6253,
  379,
  262,
  4436,
  284,
  651,
  262,
  3871,
  832,
  262,
  1755,
  13,
  198,
  198,
  18474,
  7317,
  2098,
  11,
  1912,
  319,
  10275,
  351,
  617,
  286,
  262,
  7519,
  11,
  326,
  262,
  1578,
  7973,
  6149,
  262,
  21402,
  3274,
  22225,
  290,
  7929,
  4816,
  284,
  36316,
  13,
  2102,
  11,
  21402,
  5953,
  36831,
  2269,
  861,
  402,
  2926,
  82,
  11,
  257,
  6253,
  508,
  373,
  379,
  262,
  4436,
  351,
  3126,
  21402,
  3315,
  8213